# Semantic Segmentation

## Deep Learning

Semantic segmentation using the KITTI dataset, and can be done using three methods:
- **Full-Convolutional Network [2014]** Uses a pretrained network, upsample using deconvolution, and have skip connections to improve coarseness of upsampling.
- **SegNet [2015]** Encoder-Decoder architecture
- **ResNet-DUC [2017]** Dense Upsampling and Hybrid Dilated convolution
- To improve the quality of segmentation can use something resembling FlowNet with optical flow to improve the quality of segmentation wrt. ground truth.

---
## Step 0: Import required packages

In [ ]:
## List all imports
import os
import csv
import math
import random
import time
import os.path as path
from datetime import datetime

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2

import tensorflow as tf

# Visualizations will be shown in the notebook.
%matplotlib inline 

repeat=0
import helper
from utils import tensorflow_check,load_vggmodel,model,optimizer,training

In [ ]:
## Datasets
root='kitti_dataset/'
training_dataset='kitti_dataset/training'
testing_dataset='kitti_dataset/testing'
vgg_path='vgg/'
save_sessions='sessions/'
output='output/'

In [ ]:
# Check Tensorflow installation
print("Tensorflow Version: ",tf.__version__)
tensorflow_check()

---
## Step 1: Defining Tensorflow Model

In [ ]:
# Hyperparameters and variables
image_shape=(160,576)
num_classes=2
learning_rate_value=1e-4
keep_prob_value=0.65
n_epochs=20
batch_size=5
Loss_summary=[]

In [ ]:
# Creating placeholders
label=tf.placeholder(tf.float32,[None,image_shape[0],image_shape[1],num_classes])
learning_rate=tf.placeholder(tf.float32)
keep_prob=tf.placeholder(tf.float32)

VGG-16 model with skip-connections and an Adam Optimizer to minimize cross-entropy loss

---
## Step 2: Runs

In [ ]:
## Load pretrained VGG-16 model and dataset pretrained on ImageNet
print("Getting images batches")
get_batches_fn=helper.gen_batch_function(training_dataset,image_shape)
print("Running Tensorflow session")
with tf.Session() as session:
    print("Loading pretrained VGG16 model")
    input_image,keep_prob,layer3,layer4,layer7=load_vggmodel(session,vgg_path)
    print("Creating model with skip-layers and deconvolutions")
    output_layer=model(layer3,layer4,layer7,num_classes)
    print("Running Adam Optimizer to minimize cross-entropy loss")
    logits,cross_entropy_loss,train_operation=optimizer(output_layer,label,learning_rate,num_classes)

    print("Global Variables initializer")
    session.run(tf.global_variables_initializer())
    print("Training")
    training(session,n_epochs,batch_size,get_batches_fn,train_operation,
                 cross_entropy_loss,input_image,label,keep_prob,learning_rate,keep_prob_value,learning_rate_value)
    print("Saving inference data")
    helper.save_inference_samples(save_sessions,root,session,image_shape,
                                  logits,keep_prob_value,input_image)